In [20]:
import pandas as pd
from scipy import signal
import numpy as np
from scipy.stats import shapiro, normaltest, skewtest, kurtosistest
from scipy.stats import lognorm, skewnorm, t, weibull_min, expon, beta
from scipy.stats import skew, kurtosis 


In [21]:
# Read data into a pandas dataframe
data = pd.read_csv('Historic_Index_Levels.csv')
data = data.drop([data.columns[0], data.columns[-1]], axis=1)
df = data

In [22]:
# Check the distribution of each column
for col in df.columns:
    x = pd.to_numeric(df[col], errors='coerce').dropna()
    if len(x) < 3:
        continue
    _, p_shapiro = shapiro(x)
    _, p_norm = normaltest(x)
    _, p_skew = skewtest(x)
    _, p_kurtosis = kurtosistest(x)
    
    #print(f"Column {col}:")
    #print(f"Shapiro-Wilk test p-value: {p_shapiro:.4f}")
    #print(f"Normality test p-value: {p_norm:.4f}")
    #print(f"Skewness test p-value: {p_skew:.4f}")
    #print(f"Kurtosis test p-value: {p_kurtosis:.4f}")
    

In [23]:
if p_shapiro > 0.05:
    print("The data is normally distributed")
elif p_norm > 0.05:
    print("The data is not normally distributed but is approximately normal")
elif p_skew > 0.05 and p_kurtosis > 0.05:
    print("The data is not normally distributed and is skewed and has heavy tails")
else:
    print("The data is not normally distributed and has significant skewness and/or kurtosis")


The data is not normally distributed and has significant skewness and/or kurtosis


In [24]:
# define the columns
columns = ['AU_CPI', 'AU_Cash', 'AU FI', 'US_FI_H', 'INT_FI_H', 'EM_FI_H', 'AU_PROP', 'AU_REITS', 'INT_REIT_U', 'AU EQ', 'INT_EQ_U', 'EM_EQ_U', 'GOLD_U']


In [25]:
# loop through each column
for col in columns:
    # get the data from the column
    data = df[col]
    # fit lognormal distribution
    param = lognorm.fit(data)
    lognorm_fit = lognorm(*param)
    lognorm_likelihood = np.sum(np.log(lognorm_fit.pdf(data)))
    # fit skew normal distribution
    param = skewnorm.fit(data)
    skewnorm_fit = skewnorm(*param)
    skewnorm_likelihood = np.sum(np.log(skewnorm_fit.pdf(data)))
    # fit Student's t-distribution
    param = t.fit(data)
    t_fit = t(*param)
    t_likelihood = np.sum(np.log(t_fit.pdf(data)))
    # fit Weibull distribution
    param = weibull_min.fit(data)
    weibull_fit = weibull_min(*param)
    weibull_likelihood = np.sum(np.log(weibull_fit.pdf(data)))
    # fit Exponential distribution
    param = expon.fit(data)
    exp_fit = expon(*param)
    exp_likelihood = np.sum(np.log(exp_fit.pdf(data)))
    # fit Beta distribution
    param = beta.fit(data)
    beta_fit = beta(*param)
    beta_likelihood = np.sum(np.log(beta_fit.pdf(data)))
    # find the best distribution
    likelihoods = [lognorm_likelihood, skewnorm_likelihood, t_likelihood, weibull_likelihood, exp_likelihood, beta_likelihood]
    distributions = ['lognormal', 'skew normal', 'Student\'s t', 'Weibull', 'Exponential', 'Beta']
    best_distribution = distributions[np.argmax(likelihoods)]
    # print the result
    print(f"{col}: {best_distribution} distribution")
    

AU_CPI: Beta distribution
AU_Cash: Beta distribution
AU FI: Beta distribution
US_FI_H: skew normal distribution
INT_FI_H: Beta distribution
EM_FI_H: Beta distribution
AU_PROP: Beta distribution
AU_REITS: Beta distribution
INT_REIT_U: Beta distribution
AU EQ: Beta distribution


C:\Users\ATA UMER\anaconda3\lib\site-packages\scipy\stats\_continuous_distns.py:709: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)


INT_EQ_U: Weibull distribution
EM_EQ_U: Beta distribution
GOLD_U: Beta distribution


In [26]:
# define the columns
columns = ['AU_CPI', 'AU_Cash', 'AU FI', 'US_FI_H', 'INT_FI_H', 'EM_FI_H', 'AU_PROP', 'AU_REITS', 'INT_REIT_U', 'AU EQ', 'INT_EQ_U', 'EM_EQ_U', 'GOLD_U']

# set number of simulations and years to project
n_simulations = 1000
n_years = 5

# create a dictionary to hold the results
results = {}

# loop through each column
for col in columns:
    # get the data from the column
    data = df[col]
    # fit the beta distribution to the data
    param = beta.fit(data)
    dist = beta(*param)
    # generate the simulated values
    simulated_values = dist.rvs(size=(n_simulations, n_years))
    # store the simulated values in the results dictionary
    results[col] = simulated_values.flatten()

# create a dataframe from the results dictionary
df_simulated = pd.DataFrame(results)


In [27]:
# create a dictionary to store the best distribution for each column
results = {}

# loop through each column in the original DataFrame
for col in df.columns:
    # get the data from the column
    data = df[col]
    # fit distributions and calculate likelihoods
    # ...
    # find the best distribution and store in the results dictionary
    best_distribution = distributions[np.argmax(likelihoods)]
    results[col] = best_distribution

# loop through each column in the simulated DataFrame
for col in df_simulated.columns:
    # get the data from the column
    data = df_simulated[col]
    # fit distributions and calculate likelihoods
    # ...
    # find the best distribution and compare to the original
    best_distribution = distributions[np.argmax(likelihoods)]
    if col in results:
        if best_distribution == results[col]:
            print(f"{col}: {best_distribution} distribution (same as original)")
        else:
            print(f"{col}: {best_distribution} distribution (different from original)")
    else:
        print(f"{col}: {best_distribution} distribution (not in original)")



AU_CPI: Beta distribution (same as original)
AU_Cash: Beta distribution (same as original)
AU FI: Beta distribution (same as original)
US_FI_H: Beta distribution (same as original)
INT_FI_H: Beta distribution (same as original)
EM_FI_H: Beta distribution (same as original)
AU_PROP: Beta distribution (same as original)
AU_REITS: Beta distribution (same as original)
INT_REIT_U: Beta distribution (same as original)
AU EQ: Beta distribution (same as original)
INT_EQ_U: Beta distribution (same as original)
EM_EQ_U: Beta distribution (same as original)
GOLD_U: Beta distribution (same as original)


In [32]:
# Calculate log returns for each column in the original dataframe
log_returns = np.log(df).diff().dropna()

# Calculate log returns for each column in the simulated dataframe
sim_log_returns = np.log(df_simulated).diff().dropna()

# create a list of dictionaries to store the metrics for each column
results = []
for col in df.columns:
    actual_mean = log_returns[col].mean()
    sim_mean = sim_log_returns[col].mean()
    actual_std = log_returns[col].std()
    sim_std = sim_log_returns[col].std()
    APE = abs(actual_mean - sim_mean) / actual_mean
    actual_skew = log_returns[col].skew()
    sim_skew = sim_log_returns[col].skew()
    actual_kurt = log_returns[col].kurtosis()
    sim_kurt = sim_log_returns[col].kurtosis()
    
    results.append({
        "column": col,
        "actual_mean": actual_mean,
        "sim_mean": sim_mean,
        "actual_std": actual_std,
        "sim_std": sim_std,
        "APE": APE,
        "actual_skew": actual_skew,
        "sim_skew": sim_skew,
        "actual_kurt": actual_kurt,
        "sim_kurt": sim_kurt
    })

# create a new dataframe from the list of dictionaries
results_df = pd.DataFrame(results)

# display the dataframe
results_df

,column,actual_mean,sim_mean,actual_std,sim_std,APE,actual_skew,sim_skew,actual_kurt,sim_kurt
0,AU_CPI,0.002184,-0.000079,0.002009,0.313646,1.036164,0.801188,0.040364,7.547142,-0.562085
1,AU_Cash,0.003415,0.000191,0.001767,0.519482,0.944172,-0.312046,0.016123,-0.851823,-0.254964
2,AU FI,0.004767,0.000156,0.020851,0.843555,0.967204,-0.194223,-0.021542,0.670380,-0.543118
3,US_FI_H,0.003497,-0.000095,0.021072,0.753957,1.027196,0.039365,0.002627,1.279271,-0.378386
4,INT_FI_H,0.003092,-0.000062,0.016203,0.596829,1.020041,-0.262679,0.019491,1.106978,-0.675654
5,EM_FI_H,0.005551,-0.000295,0.038722,1.037144,1.053220,-2.593286,0.021012,18.960850,-0.418860
6,AU_PROP,0.005040,0.000276,0.030318,0.727161,0.945285,-0.419229,0.026731,2.668133,-0.568315
7,AU_REITS,0.005628,0.000050,0.049951,0.856313,0.991052,-2.707241,0.011366,19.205240,-0.368644
8,INT_REIT_U,0.006961,0.000089,0.044808,1.025007,0.987145,-0.810515,0.001947,3.500958,-0.352176
9,AU EQ,0.006770,0.000141,0.039443,0.992203,0.979216,-1.177223,-0.003021,3.931315,-0.547749


In [33]:
# export dataframe to excel file
df_simulated.to_excel('Simulations.xlsx', index=False)

In [34]:
Simulations = pd.read_excel('Simulations.xlsx')
Simulations

,AU_CPI,AU_Cash,AU FI,US_FI_H,INT_FI_H,EM_FI_H,AU_PROP,AU_REITS,INT_REIT_U,AU EQ,INT_EQ_U,EM_EQ_U,GOLD_U
0,128.746337,123.007538,384.392412,325.813508,536.897236,923.940873,134327.857569,585.577922,784.262193,1673.260972,4535.499408,660.226819,731.183474
1,81.782727,291.435326,701.601440,649.768151,341.158323,249.569474,316662.375915,1120.983670,167.687669,4798.843919,2477.654781,630.835331,1317.547446
2,78.689802,314.919484,879.670591,659.733860,256.722325,922.634350,456696.568220,620.318982,796.544829,6446.920129,2521.678609,667.969107,665.575171
3,128.853555,103.046702,447.438722,699.765654,253.889689,672.547909,656522.726179,757.001375,275.865881,1375.787527,2869.996972,1450.278194,1707.038769
4,109.099112,275.497888,950.611181,204.573559,388.897125,602.873486,643896.214331,757.135168,2200.108968,4756.249837,6610.529227,1576.068514,2285.465068
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,117.956457,195.355006,594.820866,584.679423,420.244105,941.724016,619057.649905,435.536451,410.363716,1957.229874,10924.580844,1477.086976,1296.800847
4996,81.645788,327.098377,606.624940,614.279067,465.325294,285.584951,478279.926698,1013.262383,548.241970,1627.883510,12447.197733,1270.739532,729.092677
4997,72.195380,325.675812,1223.319741,650.325903,295.079790,106.371798,348374.721116,829.111868,150.693022,1801.194832,7428.909495,2306.273665,532.437386
4998,87.432365,250.518154,637.926271,512.897123,530.009121,575.124827,173912.636322,719.400433,1225.115771,3628.185373,18191.328576,1613.058404,1476.194862
